# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [181]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Ask for parameters

In [182]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [183]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Decision Variables:

*  *w* = acres of land devoted to wheat.
*  *c* = acres of land devoted to corn.
*  *b* = acres of land devoted to sugar beets.

## Auxiliary Variables:

* *w_b* = tons of wheat purchased.
* *c_b* = tons of corn purchased.
* *w_s* = tons of wheat sold.
* *c_s* = tons of corn sold.
* *b_s* = tons of sugar beets sold at the regular price.
* *b_e* = tons of sugar beets sold in excess of the quota. 


# 2. Ask for objective

In [184]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [185]:
Markdown(response2.text)

Minimize:  

150*w + 230*c + 260*b  +  216*w_b + 180*c_b - 170*w_s - 150*c_s - 36*b_s - 10*b_e 


# 3. Ask for constraints

In [186]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [187]:
Markdown(response3.text)

## Constraints:

**Land availability:**

* w + c + b ≤ 500 

**Feeding requirements and buying/selling balance:**

* 2.5w + w_b - w_s ≥ 200 
* 3c + c_b - c_s ≥ 240 

**Sugar beet quota and sales:**

* b_s ≤ 6000
* 20b - b_s - b_e = 0 

**Non-negativity of decision variables:**

* w, c, b, w_b, c_b, w_s, c_s, b_s, b_e ≥ 0 


## 3. Generate the pyomo code

In [188]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [189]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data if needed
land_available = 500
wheat_required = 200
corn_required = 240
beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of corn
model.b = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of beets

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat bought
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn bought
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.b_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (regular)
model.b_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (excess)

# Objective Function
model.cost = pyo.Objective(
    expr=150 * model.w
    + 230 * model.c
    + 260 * model.b
    + 216 * model.w_b
    + 180 * model.c_b
    - 170 * model.w_s
    - 150 * model.c_s
    - 36 * model.b_s
    - 10 * model.b_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.w + model.c + model.b <= land_available
)

model.wheat_constraint = pyo.Constraint(
    expr=2.5 * model.w + model.w_b - model.w_s >= wheat_required
)

model.corn_constraint = pyo.Constraint(
    expr=3 * model.c + model.c_b - model.c_s >= corn_required
)

model.beet_quota_constraint = pyo.Constraint(expr=model.b_s <= beet_quota)

model.beet_sales_constraint = pyo.Constraint(
    expr=20 * model.b - model.b_s - model.b_e == 0
)

# Solve the model
solver = pyo.SolverFactory("glpk")  # Choose a solver you have installed
solver.solve(model)

# Print the solution
print("Optimal Solution:")
print(f"Wheat Acreage (w): {pyo.value(model.w)}")
print(f"Corn Acreage (c): {pyo.value(model.c)}")
print(f"Sugar Beet Acreage (b): {pyo.value(model.b)}")
print(f"Wheat Bought (w_b): {pyo.value(model.w_b)}")
print(f"Corn Bought (c_b): {pyo.value(model.c_b)}")
print(f"Wheat Sold (w_s): {pyo.value(model.w_s)}")
print(f"Corn Sold (c_s): {pyo.value(model.c_s)}")
print(f"Beets Sold (Regular) (b_s): {pyo.value(model.b_s)}")
print(f"Beets Sold (Excess) (b_e): {pyo.value(model.b_e)}")
print(f"Total Cost: {pyo.value(model.cost)}")
```

In this code: 

1.  I used `pyo.ConcreteModel()` to create a concrete Pyomo model.
2.  I defined the decision variables (`w`, `c`, `b`, etc.) using `pyo.Var()` and specified that they are non-negative real numbers.
3.  The objective function and constraints are defined using Pyomo expressions (`expr`).
4.  You'll need to have a solver like GLPK installed and accessible to Python.  You can install it using `conda install -c conda-forge glpk`
5.  Replace the sample data (`land_available`, `wheat_required`, `corn_required`, `beet_quota`) with your actual data.
6.  Finally, the code solves the model and prints the optimal solution.


## 4. Input problem data and try running the generated code

In [190]:
import pyomo.environ as pyo

# Sample Data - Replace with your actual data if needed
land_available = 500
wheat_required = 200
corn_required = 240
beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of corn
model.b = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of beets

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat bought
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn bought
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.b_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (regular)
model.b_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (excess)

# Objective Function
model.cost = pyo.Objective(
    expr=150 * model.w
    + 230 * model.c
    + 260 * model.b
    + 216 * model.w_b
    + 180 * model.c_b
    - 170 * model.w_s
    - 150 * model.c_s
    - 36 * model.b_s
    - 10 * model.b_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.w + model.c + model.b <= land_available
)

model.wheat_constraint = pyo.Constraint(
    expr=2.5 * model.w + model.w_b - model.w_s >= wheat_required
)

model.corn_constraint = pyo.Constraint(
    expr=3 * model.c + model.c_b - model.c_s >= corn_required
)

model.beet_quota_constraint = pyo.Constraint(expr=model.b_s <= beet_quota)

model.beet_sales_constraint = pyo.Constraint(
    expr=20 * model.b - model.b_s - model.b_e == 0
)

# Solve the model
solver = pyo.SolverFactory("glpk")  # Choose a solver you have installed
solver.solve(model)

# Print the solution
print("Optimal Solution:")
print(f"Wheat Acreage (w): {pyo.value(model.w)}")
print(f"Corn Acreage (c): {pyo.value(model.c)}")
print(f"Sugar Beet Acreage (b): {pyo.value(model.b)}")
print(f"Wheat Bought (w_b): {pyo.value(model.w_b)}")
print(f"Corn Bought (c_b): {pyo.value(model.c_b)}")
print(f"Wheat Sold (w_s): {pyo.value(model.w_s)}")
print(f"Corn Sold (c_s): {pyo.value(model.c_s)}")
print(f"Beets Sold (Regular) (b_s): {pyo.value(model.b_s)}")
print(f"Beets Sold (Excess) (b_e): {pyo.value(model.b_e)}")
print(f"Total Cost: {pyo.value(model.cost)}")

Optimal Solution:
Wheat Acreage (w): 120.0
Corn Acreage (c): 80.0
Sugar Beet Acreage (b): 300.0
Wheat Bought (w_b): 0.0
Corn Bought (c_b): 0.0
Wheat Sold (w_s): 99.9999999999999
Corn Sold (c_s): 0.0
Beets Sold (Regular) (b_s): 6000.0
Beets Sold (Excess) (b_e): 0.0
Total Cost: -118599.99999999999


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [191]:
print(response.text)

## Decision Variables:

*  *w* = acres of land devoted to wheat.
*  *c* = acres of land devoted to corn.
*  *b* = acres of land devoted to sugar beets.

## Auxiliary Variables:

* *w_b* = tons of wheat purchased.
* *c_b* = tons of corn purchased.
* *w_s* = tons of wheat sold.
* *c_s* = tons of corn sold.
* *b_s* = tons of sugar beets sold at the regular price.
* *b_e* = tons of sugar beets sold in excess of the quota. 



In [192]:
print(response2.text)

Minimize:  

150*w + 230*c + 260*b  +  216*w_b + 180*c_b - 170*w_s - 150*c_s - 36*b_s - 10*b_e 



In [193]:
print(response3.text)

## Constraints:

**Land availability:**

* w + c + b ≤ 500 

**Feeding requirements and buying/selling balance:**

* 2.5w + w_b - w_s ≥ 200 
* 3c + c_b - c_s ≥ 240 

**Sugar beet quota and sales:**

* b_s ≤ 6000
* 20b - b_s - b_e = 0 

**Non-negativity of decision variables:**

* w, c, b, w_b, c_b, w_s, c_s, b_s, b_e ≥ 0 



In [194]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data if needed
land_available = 500
wheat_required = 200
corn_required = 240
beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of corn
model.b = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of beets

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat bought
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn bought
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.b_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (regular)
model.b_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of beets sold (excess)

# Objective Function
model.cost = pyo.Objective(
    expr=150 * model.w
    + 230 * model.c
    + 260